# Python code for final project PHZ5305: Interaction matrix elements and Hartree-Fock (Nov. 29, 2024)

## Introduction

This code is written for the final proejct of PHZ5305 (Nuclear Physics 1), which will provide the tool for the read the list of single particle states and me2b generated, and solves Hartree-Fock Equation.

Additionally, this code will optimize the me2b values into the value that simultaneously gives binding energies of the $^4 He$, $^{12} C$, $^{16} O$.

## Part 1: Import necessary packages.

`numpy` provides mathematical tools for calculation, such as `power`, `list`, and `array`.

`pandas` provides the reading csv file that contains the experimental informations that will provide the `me2b`.

`scipy` provides the optimization feature of this code.

`random` is imported for giving a random value assigned for the other `me2b` values.

In [1]:
import numpy as np
import pandas as pd
import scipy as scp
import scipy.sparse as sp
from scipy.optimize import minimize
import random
import sys

## Part 2: Class for Generate single-particle states, two-body matrix element calculation, and solve Hartree-Fock.

In this block, the Hartree-Fock equation will be solved via algorithm that intorudced at the class (Oct. 14, 2024) with single-particle states and two-body matrix elements. 

`__init__` gives initial definition for this class

`SP_States` generates `dictionary` of SP states, $\pi 0s1/2$, $\nu 0s1/2$, $\pi 0p3/2$, $\nu 0p3/2$, $\pi 0p1/2$, $\nu 0p 1/2$ in `{index: [n, l, j, mj, tz]}` format.  

`me2b` gets 4 index values for $\alpha$, $\beta$, $\gamma$, and $\delta$.

`Solve_HF` solves Hartree-Fock equation with algorithm

In [2]:
class HF_Machina:
    # Initial definition of states
    def __init__(self):
        self.SP_States()
        self.TB_States()
        
    # Generate Dictionaries of Single-Particle States up to 16O
    def SP_States(self):
        
        # Initialize the important parameters which will be applied to loop #
        list_tz      = [-1/2, 1/2]
        n            = 0
        list_l       = [0, 1]
        dict_list_j  = {
            0: [1/2],
            1: [3/2, 1/2]
        }
        
        # Set the index and empty dictionary
        index        = 0 
        dict_SP      = {}
        
        # Loop for fullfill the dictionary
        for l in list_l:
            for tz in list_tz:
                for j in dict_list_j[l]:
                    mj = -1*j
                    while mj < j+1:
                        dict_SP.update({index: [n,l,j,mj,tz]})
                        index += 1
                        mj += 1
        
        # Output
        self.dict_SP = dict_SP
        return dict_SP

    # Generate List of Two-body states in One-body basis (a,b) for Jpi = 0+
    def TB_States(self):
        
        #Initialize
        NSP = 16
        
        list_TB_ind = []
        
        for a in range(NSP):
            for b in range(NSP):
                # Imply possible conditions to be Jpi = 0+ 
                # ja = jb
                j_cond  = (self.dict_SP[a][2] == self.dict_SP[b][2])
                # mja = -mjb
                mj_cond = (self.dict_SP[a][3] == -1*self.dict_SP[b][3])
                # la = lb (since only possible l = 0 or 1)
                l_cond  = (self.dict_SP[a][1] == self.dict_SP[b][1])
                if j_cond and mj_cond and l_cond: list_TB_ind.append((a,b))
        
        list_TB_ind = list({frozenset(pair) for pair in list_TB_ind})
        list_TB_ind = [tuple(sorted(pair)) for pair in list_TB_ind]
        
        self.list_TB = list_TB_ind
        return list_TB_ind
        
    
    # Code for solving Hartree-Fock equation from core 12C 
    # A is the atomic number of nucleus 
    # Z is the charge of nucleus 
    # input_me2b is a me2b input with form of list [(a,b,c,d,v)]
    def Solve_HF(self, A, Z, input_me2b):
        
        # Prepare Initial Value
        self.A_c      = 12
        self.Z_c      = 6
        self.A        = A
        self.Z        = Z
        NSP           = len(self.dict_SP)
        
        # Prepare me2b
        self.dict_me2b = {(me2b[0], me2b[1], me2b[2], me2b[3]): me2b[4] for me2b in input_me2b}
        
        # Computing single-particle Hamiltonian (SPH)
        SPH = np.array([self.onebody(sp_ind) for sp_ind in range(len(self.dict_SP))])
        
        # Initialize the Coefficient Matrix Coe and Density Matrix Rho 
        Coe = np.eye(NSP)
        Rho = np.zeros((NSP, NSP), dtype=object)

        for i_gam in range(NSP):
            for i_del in range(NSP):
                Rho[i_gam, i_del] = sum(Coe[i_gam,i]*Coe[i_del,i] for i in range(self.A))
        
        # Now, we will do the algorithm that will be iterated until it converges at somewhere or its maximum iteration number
        # Set initial value for iteration
        maxiter, epsl = 100, 1.0e-10
        diff, i_count = 1.0, 0
        
        # Set Energies 
        oldE = np.zeros(NSP)
        newE = np.zeros(NSP)
        
        # Iterating alogrithm!
        while i_count < maxiter and diff > epsl:
            
            # Make Hartree-Fock Matrix
            MHF = np.zeros((NSP, NSP))
            
            for i_alp in range(NSP):
                for i_bet in range(NSP):
                    if self.dict_SP[i_alp][1] != self.dict_SP[i_bet][1]: continue
                    sum_me2b = sum(
                        Rho[i_gam, i_del]*self.dict_me2b.get((i_alp,i_gam,i_bet,i_del),0)
                        for i_gam in range(NSP)
                        for i_del in range(NSP)
                        if (self.dict_SP[i_alp][3]+self.dict_SP[i_gam][3] == self.dict_SP[i_bet][3]+self.dict_SP[i_del][3]) 
                        and (self.dict_SP[i_alp][4]+self.dict_SP[i_gam][4] == self.dict_SP[i_bet][4]+self.dict_SP[i_del][4])
                    )
                    MHF[i_alp,i_bet] = sum_me2b + (SPH[i_alp] if i_alp == i_bet else 0)
                
            # Diagonalize and get the eigenstates of MHF
            Eeig, Coe = np.linalg.eigh(np.array(MHF))
            Rho = np.zeros((NSP, NSP))
            
            # Update Rho
            for i_gam in range(NSP):
                for i_del in range(NSP):
                    Rho[i_gam, i_del] = sum(Coe[i_gam,i]*Coe[i_del,i] for i in range(self.A))
            
            # Get new Energies and calculate convergence
            newE     = np.array([e for e in Eeig])
            diff     = np.sum(np.abs(newE-oldE)/NSP)
            oldE     = newE
            i_count += 1
            
            # Print results
            
            #print("Single-particle energies, ordering may have changed")
            #for i in range(NSP):
            #    print('{0} {1:.4f}'.format(i,oldE[i]))
                
            #print('{0:.4f}'.format(sum(oldE[i] for i in range(self.A))))
            return sum(oldE[i] for i in range(self.A))
        
    # Single Particle Hamiltonian (SPH)
    def onebody(self, SP_ind):
        e_gap = 41.5*np.power(self.A_c, -0.333)
        n, l  = self.dict_SP[SP_ind][0], self.dict_SP[SP_ind][1]
        return e_gap*(2*n + l + 3/2)
    

## Part 3: Set the initial value of me2b for code

In this block, it will generate initial TBME as a list of (a,b,c,d,v), where using the method $\langle j_1 j_2 |V| j_1 j_2\rangle = BE(CS\pm 2 , j_1 j_2) - BE(CS) - 2 \epsilon^{HF}$, as given as <Shell Model from a Practitioner’s Point of View> by Hubert Grawe. 

This section needs to be refined again.

In [3]:
def generate_TBME(dict_SP, list_TB):
    
    BE_ds = pd.read_csv('atomicMassData.csv')
    e_gap = 41.5*np.power(12,-1/3)
    A_core = 12
    Z_core = 6
    A_gap = {
        1.5:-2,
        0.5:2
    }
    Z_gap = {
        1:2,
        0:1,
        -1:0
    }
        
    Matrx_tbme = np.zeros((16,16,16,16))
    list_tbme  = []
    for i in list_TB:
        for j in list_TB:
            tbme = (random.randint(0,1000)-500)
            Matrx_tbme[i[0]][i[1]][j[0]][j[1]] = tbme/1000
            if dict_SP[i[0]][4]+dict_SP[i[1]][4] != dict_SP[j[0]][4] + dict_SP[j[1]][4] : continue
            if i != j : continue
            if dict_SP[i[0]][2]==0 : continue
            ji, tzi = dict_SP[i[0]][2], dict_SP[i[0]][4]+dict_SP[i[1]][4]
            CSpm2   = BE_ds[(BE_ds['Z']==Z_core+Z_gap.get(tzi)*A_gap.get(ji)/2) & (BE_ds['A']==A_core+A_gap.get(ji))]['BE'].tolist()[0]
            CS      = BE_ds[(BE_ds['Z']==6) & (BE_ds['A']==12)]['BE'].tolist()[0]
            tbme    = (A_core+A_gap.get(ji))*CSpm2-A_core*CS-2*e_gap*1000
            Matrx_tbme[i[0]][i[1]][j[0]][j[1]] = tbme/1000
                
    Matrx_tbme = (Matrx_tbme+Matrx_tbme.T)/2
    
    """
    for i in range(16):
        for j in range(16):
            for k in range(16):
                for l in range(16):
                    list_tbme.append((i,j,k,l,Matrx_tbme[i][j][k][l]))
    """
    
    for i in list_TB:
        for j in list_TB:
            list_tbme.append((i[0],i[1],j[0],j[1],Matrx_tbme[i[0]][i[1]][j[0]][j[1]]))
            
    #print(list_tbme)
                    
    return list_tbme

## Part 4: Set a function for optimization

In [4]:
def minimize_this(x):
    exp_energies = {
        '4He':-28.30,
        '12C':-92.16,
        '16O':-127.62
    }
        
    HF = HF_Machina()
    
    list_tbme = generate_TBME(HF.dict_SP, HF.list_TB)
    
    list_input_tbme = []
    
    for i, (a,b,c,d,_) in enumerate(list_tbme):
        list_input_tbme.append((a,b,c,d,x[i]))
    
    e_4He = HF.Solve_HF(4,2,list_input_tbme)
    e_12C = HF.Solve_HF(12,6,list_input_tbme)
    e_16O = HF.Solve_HF(16,8,list_input_tbme)
    
    error = (e_4He - exp_energies['4He'])**2 + \
            (e_12C - exp_energies['12C'])**2 + \
            (e_16O - exp_energies['16O'])**2
    
    error = float(error)
    
    %clear 5
    """
    print("\n 4He energy : "+str(e_4He)+
          "\n 12C energy : "+str(e_12C)+
          "\n 16O energy : "+str(e_16O)+
          "\n error      : "+str(error)+
          "\n -------------------------", end='')
    """

    print("\n 4He energy : "+str(e_4He)+
          "\n 12C energy : "+str(e_12C)+
          "\n 16O energy : "+str(e_16O)+
          "\n error      : "+str(error)+
          "\n -------------------------", end='')
    sys.stdout.flush()
    
    return error

In [5]:
if __name__=='__main__':
    HF = HF_Machina()
    
    tbme0 = generate_TBME(HF.dict_SP, HF.list_TB)
    
    x0 = [t[4] for t in tbme0]
    
    result = minimize(minimize_this, x0, method='SLSQP', tol=1e-6)


 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.3122314063996
 12C energy : 288.1548357387128
 16O energy : 329.95174551011746
 error      : 360034.93503602024
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.1548357

 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.3122314063996
 12C energy : 288.1548357387128
 16O energy : 329.95174551011746
 error      : 360034.93503602024
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436

 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.95174551011746
 error      : 360034.935022373
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.31223139149843

 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.95174551011746
 error      : 360034.935022373
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.31223139149843

 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.9517454952163
 error      : 360034.9350087363
 -------------------------
 4He energy : 49.312231391498436
 12C energy : 288.15483572381163
 16O energy : 329.95174551011746
 error      : 360034.935022373
 -------------------------
 4He energy : 49.31223139149843

...

 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.29999999275563
 12C energy : -92.15999999226551
 16O energy : -127.61999999268318
 error      : 1.6583913450759904e-16
 ---------------

 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 ---------------

 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 ---------------

 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 ---------------

 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.15999999226551
 16O energy : -127.61999999268318
 error      : 1.7198467617728421e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 -------------------------
 4He energy : -28.30000000765679
 12C energy : -92.16000000716667
 16O energy : -127.62000000758434
 error      : 1.6750980111693975e-16
 ---------------

In [9]:
list_input_tbme=[]
for i, (a,b,c,d,_) in enumerate(tbme0):
        list_input_tbme.append((a,b,c,d,result.x[i]))
        
print(list_input_tbme)

[(2, 3, 2, 3, -31.623673493066974), (2, 3, 11, 12, 0.04700000000000254), (2, 3, 1, 2, 0.10250000000000131), (2, 3, 0, 3, -0.24099999999999835), (2, 3, 4, 13, 0.21700000000000505), (2, 3, 0, 1, 0.21650000000000008), (2, 3, 5, 12, -0.18550000000000763), (2, 3, 7, 10, -0.20150000000000307), (2, 3, 8, 9, 0.23250000000000395), (2, 3, 10, 13, 0.21449999999999975), (2, 3, 5, 6, 0.025500000000001), (2, 3, 9, 14, 0.23149999999999793), (2, 3, 14, 15, 0.18400000000000163), (2, 3, 8, 15, 0.08000000000000035), (2, 3, 6, 11, 0.19500000000000037), (2, 3, 4, 7, 0.015499999999998978), (11, 12, 2, 3, -0.0275), (11, 12, 11, 12, -37.567802039252804), (11, 12, 1, 2, 0.1595), (11, 12, 0, 3, 0.158), (11, 12, 4, 13, 0.1975), (11, 12, 0, 1, -0.0835), (11, 12, 5, 12, -0.222), (11, 12, 7, 10, 0.128), (11, 12, 8, 9, 0.1305), (11, 12, 10, 13, -0.206), (11, 12, 5, 6, 0.1005), (11, 12, 9, 14, -0.2115), (11, 12, 14, 15, -0.104), (11, 12, 8, 15, -0.199), (11, 12, 6, 11, -0.092), (11, 12, 4, 7, -0.119), (1, 2, 2, 3, 0.